In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24261
2,Huelva,Confirmados PDIA 14 días,986
3,Huelva,Tasa PDIA 14 días,"192,13905723249604"
4,Huelva,Confirmados PDIA 7 días,308
5,Huelva,Total Confirmados,24489
6,Huelva,Curados,14130
7,Huelva,Fallecidos,274


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24261.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6481.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1498 personas en los últimos 7 días 

Un positivo PCR cada 462 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,438.0,1.0,2.0,3366.0,29.708853,59.417706,1.0
Trigueros,224.0,5.0,6.0,7862.0,63.597049,76.316459,1.0
Villanueva de los Castillejos,145.0,4.0,9.0,2825.0,141.592920,318.584071,-1.0
San Bartolomé de la Torre,191.0,3.0,4.0,3761.0,79.766020,106.354693,-1.0
Villablanca,167.0,4.0,11.0,2885.0,138.648180,381.282496,-1.0
Alosno,119.0,1.0,3.0,3933.0,25.425884,76.277651,-1.0
Nerva,151.0,1.0,7.0,5169.0,19.346102,135.422712,-1.0
Palos de la Frontera,439.0,5.0,22.0,11742.0,42.582184,187.361608,-1.0
Ayamonte,1178.0,12.0,39.0,21104.0,56.861259,184.799090,-2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,167.0,4.0,11.0,2885.0,138.648180,381.282496,-1.0
Escacena del Campo,136.0,3.0,8.0,2287.0,131.176213,349.803236,0.0
Villanueva de los Castillejos,145.0,4.0,9.0,2825.0,141.592920,318.584071,-1.0
Almonte,826.0,28.0,78.0,24507.0,114.253071,318.276411,0.0
Cartaya,915.0,31.0,62.0,20083.0,154.359408,308.718817,0.0
Lepe,1964.0,21.0,86.0,27880.0,75.322812,308.464849,0.0
Bollullos Par del Condado,591.0,16.0,43.0,14387.0,111.211510,298.880934,0.0
Isla Cristina,1971.0,15.0,62.0,21393.0,70.116393,289.814425,-10.0
Punta Umbría,647.0,11.0,36.0,15355.0,71.637903,234.451319,-3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,414.0,1.0,13.0,12750.0,7.843137,101.960784,-9.0,0.076923
Sierra de Huelva-Andévalo Central,2299.0,6.0,44.0,67391.0,8.903266,65.290617,-28.0,0.136364
Nerva,151.0,1.0,7.0,5169.0,19.346102,135.422712,-1.0,0.142857
Palma del Condado (La),960.0,2.0,11.0,10801.0,18.516804,101.842422,0.0,0.181818
San Juan del Puerto,355.0,1.0,5.0,9411.0,10.625863,53.129317,0.0,0.200000
Palos de la Frontera,439.0,5.0,22.0,11742.0,42.582184,187.361608,-1.0,0.227273
Isla Cristina,1971.0,15.0,62.0,21393.0,70.116393,289.814425,-10.0,0.241935
Lepe,1964.0,21.0,86.0,27880.0,75.322812,308.464849,0.0,0.244186
Cerro de Andévalo (El),135.0,1.0,4.0,2327.0,42.973786,171.895144,0.0,0.250000
